In [1]:
# 필요 패키지 로드
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fonm
from matplotlib import font_manager, rc
mpl.rcParams['axes.unicode_minus'] = False
font_path = 'C:/Windows/Fonts/정직_1.ttf'
font = fonm.FontProperties(fname=font_path).get_name()
rc('font', family=font)
import pandas as pd
import seaborn as sns

In [7]:
# 로지스틱 모델 로드 및 테스트
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import joblib
lr = LogisticRegression()

In [15]:
# 기존에 처리한 데이터 전처리 코드 로드
df = pd.read_csv('C:\\Workspace\\python\\빅데이터 지능형서비스 개발 팀프로젝트(21.9.27~10.6)\\Project-1\\조정범\\Data\\모델제작용데이터셋(헬스케어).csv')
for i in df.columns:
    df[i] = df[i].astype('category')

print(f"전체환자 데이터 수 : {len(df)} / 입원환자 데이터 수 : {len(df[df['서식코드'] == 1])} / 비입원환자 데이터 수 : {len(df[df['서식코드'] == 0])}")

# 출처 : https://g0n1.tistory.com/34
# 클래스 불균형 처리를 해주는 imbalanced-learn를 호출
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# ros, rus 객체
ros = RandomOverSampler()
rus = RandomUnderSampler()

# Under 샘플링
# Under 샘플링
undersampled_data, undersampled_label = rus.fit_resample(df[['성별코드', '연령대코드', '시도코드', '진료과목코드', '주상병코드', '부상병코드','심결가산율']].copy(), df[['서식코드']].copy())
undersampled_data = pd.DataFrame(undersampled_data, columns =['성별코드', '연령대코드', '시도코드', '진료과목코드', '주상병코드', '부상병코드','심결가산율'])
undersampled_data['서식코드'] = undersampled_label

df_train1 = undersampled_data[['서식코드']].copy() # 테스트로 사용할 컬럼들
def dummy_data(data, columns):
    for column in columns:
        data = pd.concat([data, pd.get_dummies(data[column], prefix = column)], axis=1)
        data = data.drop(column, axis=1)
    return data
dummy_columns = ['서식코드']
df_train1_dummy = dummy_data(undersampled_data, dummy_columns)
print(f"전체환자 데이터 수 : {len(undersampled_data)} / 입원환자 데이터 수 : {len(undersampled_data[undersampled_data['서식코드'] == 1])} / 비입원환자 데이터 수 : {len(undersampled_data[undersampled_data['서식코드'] == 0])}")

# y값 원-핫 인코딩
# 적용
# X = df_train1_dummy.iloc[:, :-2]
# y = df_train1_dummy.iloc[:, 7:]
# 미적용
X = undersampled_data.iloc[:, :-1]
y = undersampled_data.iloc[:, -1:]

# # Over 샘플링
# oversampled_data, oversampled_label = ros.fit_resample(df[['성별코드', '연령대코드', '시도코드', '진료과목코드', '주상병코드', '부상병코드','심결가산율']].copy(), df[['서식코드']].copy())
# oversampled_data = pd.DataFrame(oversampled_data, columns = ['성별코드', '연령대코드', '시도코드', '진료과목코드', '주상병코드', '부상병코드','심결가산율'])
# oversampled_data['서식코드'] = oversampled_label

# df_train1 = oversampled_data[['서식코드']].copy() # 테스트로 사용할 컬럼들
# def dummy_data(data, columns):
#     for column in columns:
#         data = pd.concat([data, pd.get_dummies(data[column], prefix = column)], axis=1)
#         data = data.drop(column, axis=1)
#     return data
# dummy_columns = ['서식코드']
# df_train1_dummy = dummy_data(oversampled_data, dummy_columns)
# print(f"전체환자 데이터 수 : {len(oversampled_data)} / 입원환자 데이터 수 : {len(oversampled_data[oversampled_data['서식코드'] == 1])} / 비입원환자 데이터 수 : {len(oversampled_data[oversampled_data['서식코드'] == 0])}")


# y값 원-핫 인코딩
# 적용
# X = df_train1_dummy.iloc[:, :-2]
# y = df_train1_dummy.iloc[:, 7:]
# # 미적용
# X = oversampled_data.iloc[:, :-1]
# y = oversampled_data.iloc[:, -1:]


전체환자 데이터 수 : 13178345 / 입원환자 데이터 수 : 283962 / 비입원환자 데이터 수 : 12894383
전체환자 데이터 수 : 567924 / 입원환자 데이터 수 : 283962 / 비입원환자 데이터 수 : 283962


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 7)

In [17]:
# X_train
y_train

,서식코드
7891,0
398837,1
461871,1
506387,1
109346,0
...,...
522482,1
275384,0
293919,1
496550,1


In [18]:
lr.fit(X_train, y_train)

C:\Users\Bestc\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LogisticRegression()

In [23]:
# 학습데이터에 대한 성능평가
print(confusion_matrix(y_train, lr.predict(X_train)))
print(classification_report(y_train, lr.predict(X_train)))

[[185272  41898]
 [ 49912 177257]]
              precision    recall  f1-score   support

           0       0.79      0.82      0.80    227170
           1       0.81      0.78      0.79    227169

    accuracy                           0.80    454339
   macro avg       0.80      0.80      0.80    454339
weighted avg       0.80      0.80      0.80    454339



In [24]:
# 테스트 데이터에 대한 성능평가
print(confusion_matrix(y_test, lr.predict(X_test)))
print(classification_report(y_test, lr.predict(X_test)))

[[46470 10322]
 [12166 44627]]
              precision    recall  f1-score   support

           0       0.79      0.82      0.81     56792
           1       0.81      0.79      0.80     56793

    accuracy                           0.80    113585
   macro avg       0.80      0.80      0.80    113585
weighted avg       0.80      0.80      0.80    113585



In [26]:
from sklearn.metrics import accuracy_score
y_pred = lr.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(classification_report(y_test, y_pred))
print(f'Accuracy : {accuracy*100}')

              precision    recall  f1-score   support

           0       0.79      0.82      0.81     56792
           1       0.81      0.79      0.80     56793

    accuracy                           0.80    113585
   macro avg       0.80      0.80      0.80    113585
weighted avg       0.80      0.80      0.80    113585

Accuracy : 80.20161112822996
